In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb
from pathlib import Path

In [3]:
pd.options.display.max_columns = 50

## Setup the path

In [4]:
path = Path('/kaggle/m5_forecasting/')
assert(path.exists())

In [5]:
def ls(self):
    return list(self.iterdir())
setattr(Path, 'ls', ls)

In [6]:
path.ls()

[PosixPath('/kaggle/m5_forecasting/sales_train_validation.csv'),
 PosixPath('/kaggle/m5_forecasting/m5_model_0.lgb'),
 PosixPath('/kaggle/m5_forecasting/m5_model_1.lgb'),
 PosixPath('/kaggle/m5_forecasting/m5_model_3.lgb'),
 PosixPath('/kaggle/m5_forecasting/m5_best_2.pth'),
 PosixPath('/kaggle/m5_forecasting/m5_best_1.pth'),
 PosixPath('/kaggle/m5_forecasting/walmartTrends0.csv'),
 PosixPath('/kaggle/m5_forecasting/m5_model_2.lgb'),
 PosixPath('/kaggle/m5_forecasting/m5_model.lgb'),
 PosixPath('/kaggle/m5_forecasting/calendar.csv'),
 PosixPath('/kaggle/m5_forecasting/sample_submission.csv'),
 PosixPath('/kaggle/m5_forecasting/m5_model_4.lgb'),
 PosixPath('/kaggle/m5_forecasting/m5_dt'),
 PosixPath('/kaggle/m5_forecasting/sell_prices.csv')]

## Read Data

In [7]:
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float16" }
CAL_DTYPES = {"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "uint8", 'snap_TX': 'uint8', 'snap_WI': 'uint8' }

In [8]:
sales_train_validation = pd.read_csv(path/"sales_train_validation.csv", nrows=10)

In [9]:
def read_data():
    prices = pd.read_csv(path/"sell_prices.csv", dtype = PRICE_DTYPES)
    cal = pd.read_csv(path/"calendar.csv", dtype = CAL_DTYPES)
    return prices, cal

In [10]:
prices, cal = read_data()

In [11]:
cal.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [12]:
prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000


#### Pre-process calendar

In [12]:
cal["date"] = pd.to_datetime(cal["date"], infer_datetime_format=True)

In [13]:
def convert_to_type(df, cols, dt_type):
    for type_name in cols:
        print(type_name)
        df[type_name] = df[type_name].astype(dt_type)

In [14]:
def convert_uint8(df, cols):
    convert_to_type(df, cols, "uint8")

In [15]:
def convert_float16(df, cols):
    convert_to_type(df, cols, "float16")

In [16]:
def numericalize(df, type_map):
    for col, col_dtype in type_map.items():
        if col_dtype == "category":
            df[col] = df[col].cat.codes.astype('int16')
            df[col] -= df[col].min()

In [17]:
numericalize(prices, PRICE_DTYPES)
numericalize(cal, CAL_DTYPES)

In [18]:
cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int16         
 2   weekday       1969 non-null   int16         
 3   wday          1969 non-null   int16         
 4   month         1969 non-null   int16         
 5   year          1969 non-null   int16         
 6   d             1969 non-null   object        
 7   event_name_1  1969 non-null   int16         
 8   event_type_1  1969 non-null   int16         
 9   event_name_2  1969 non-null   int16         
 10  event_type_2  1969 non-null   int16         
 11  snap_CA       1969 non-null   uint8         
 12  snap_TX       1969 non-null   uint8         
 13  snap_WI       1969 non-null   uint8         
dtypes: datetime64[ns](1), int16(9), object(1), uint8(3)
memory usage: 71.3+ KB


In [19]:
cal['snap_CA'].unique()

array([0, 1], dtype=uint8)

In [20]:
convert_uint8(cal, ['wday', 'month', 'weekday', 'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2', 
                    'snap_CA', 'snap_TX', 'snap_WI'])

wday
month
weekday
event_name_1
event_name_2
event_type_1
event_type_2
snap_CA
snap_TX
snap_WI


In [21]:
cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int16         
 2   weekday       1969 non-null   uint8         
 3   wday          1969 non-null   uint8         
 4   month         1969 non-null   uint8         
 5   year          1969 non-null   int16         
 6   d             1969 non-null   object        
 7   event_name_1  1969 non-null   uint8         
 8   event_type_1  1969 non-null   uint8         
 9   event_name_2  1969 non-null   uint8         
 10  event_type_2  1969 non-null   uint8         
 11  snap_CA       1969 non-null   uint8         
 12  snap_TX       1969 non-null   uint8         
 13  snap_WI       1969 non-null   uint8         
dtypes: datetime64[ns](1), int16(2), object(1), uint8(10)
memory usage: 57.8+ KB


In [22]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    int16  
 1   item_id     int16  
 2   wm_yr_wk    int16  
 3   sell_price  float16
dtypes: float16(1), int16(3)
memory usage: 52.2 MB


In [23]:
pred_days = 28
max_lags = pred_days * 2 + 1
print('max_lags', max_lags)
num_cols = [c for c in pd.read_csv(path/"sales_train_validation.csv", nrows=2).columns if c.find('d_') == 0]
tr_last = len(num_cols)
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
# For more training data use a lower value
FIRST_DAY=1

max_lags 57


In [24]:
# attr = ['Dayofyear','Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
boolean_attrs = ['Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
date_features = {
    "wday": "weekday",
    "week": "weekofyear",
    "month": "month",
    "quarter": "quarter",
    "year": "year",
    "mday": "day"
}

In [25]:
def prepare_date_cols(dt):
    for date_feature_name, date_feature_func in date_features.items():
        if date_feature_name in dt.columns:
            dt[date_feature_name] = dt[date_feature_name].astype('int16')
        else:
            dt[date_feature_name] = getattr(dt['date'].dt, date_feature_func).astype('int16')
    for ba in boolean_attrs:
        dt[ba] = getattr(dt['date'].dt, ba.lower()).astype('uint8')

In [26]:
def read_dt(is_train = True, nrows = None, first_day = 1200):
    start_day = max(1 if is_train else tr_last - max_lags, first_day)
    print('start_day', start_day)
    dtype = {num: 'float32' for num in num_cols}
    dtype.update({cat: 'category' for cat in catcols if cat != 'id'})
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    dt = pd.read_csv(path/"sales_train_validation.csv", nrows=nrows, usecols = catcols + numcols, dtype=dtype)
    for col in catcols:
        if col != 'id':
            dt[col] = dt[col].cat.codes.astype('int16')
            dt[col] -= dt[col].min()
    if not is_train:
        for day in range(tr_last + 1, tr_last + 1 + pred_days):
            dt[f'd_{day}'] = np.nan
            
    dt = dt.melt(id_vars=catcols, value_vars=[col for col in dt.columns if col.startswith("d_")], var_name='d', value_name='sales')
    dt = dt.merge(cal, on='d', copy=False)
    dt = dt.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'], copy=False)
    
    prepare_date_cols(dt)
    convert_uint8(dt, ['dept_id', 'store_id', 'cat_id', 'state_id', 'wday', 'month', 'week', 'quarter', 'mday'])
    return dt

In [27]:
%%time

dt = read_dt(first_day=FIRST_DAY)

start_day 1
dept_id
store_id
cat_id
state_id
wday
month
week
quarter
mday
CPU times: user 39.8 s, sys: 3.08 s, total: 42.9 s
Wall time: 42.9 s


In [28]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46027957 entries, 0 to 46027956
Data columns (total 31 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                object        
 1   item_id           int16         
 2   dept_id           uint8         
 3   store_id          uint8         
 4   cat_id            uint8         
 5   state_id          uint8         
 6   d                 object        
 7   sales             float32       
 8   date              datetime64[ns]
 9   wm_yr_wk          int16         
 10  weekday           uint8         
 11  wday              uint8         
 12  month             uint8         
 13  year              int16         
 14  event_name_1      uint8         
 15  event_type_1      uint8         
 16  event_name_2      uint8         
 17  event_type_2      uint8         
 18  snap_CA           uint8         
 19  snap_TX           uint8         
 20  snap_WI           uint8         
 21  sell_p

## Create features

In [29]:
def create_features(dt):
    lags = [7, 28]
    lag_cols = [f'sales_lag_{lag}' for lag in lags]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[['id', 'sales']].groupby('id')['sales'].shift(lag)
        
    for win in lags:
        for lag, lag_col in zip(lags, lag_cols):
            grouped = dt[['id', lag_col]].groupby('id')[lag_col]
            dt[f'rmean_{lag}_{win}'] = grouped.transform(lambda x : x.rolling(win).mean())
            
    convert_float16(dt, ['sales_lag_7', 'sales_lag_28', 'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28'])

In [30]:
%%time
create_features(dt)

sales_lag_7
sales_lag_28
rmean_7_7
rmean_28_7
rmean_7_28
rmean_28_28
CPU times: user 1min 33s, sys: 5.7 s, total: 1min 39s
Wall time: 1min 39s


In [31]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46027957 entries, 0 to 46027956
Data columns (total 37 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                object        
 1   item_id           int16         
 2   dept_id           uint8         
 3   store_id          uint8         
 4   cat_id            uint8         
 5   state_id          uint8         
 6   d                 object        
 7   sales             float32       
 8   date              datetime64[ns]
 9   wm_yr_wk          int16         
 10  weekday           uint8         
 11  wday              uint8         
 12  month             uint8         
 13  year              int16         
 14  event_name_1      uint8         
 15  event_type_1      uint8         
 16  event_name_2      uint8         
 17  event_type_2      uint8         
 18  snap_CA           uint8         
 19  snap_TX           uint8         
 20  snap_WI           uint8         
 21  sell_p

In [32]:
dt.dropna(inplace = True)

In [ ]:
dt[dt.date == '2015-01-31']

## Training Preparation

In [ ]:
dt.columns

In [ ]:
cat_feats = ['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'event_name_1', 
             'event_type_1', 'event_name_2', 'event_type_2']
ignore_cols = ['id', 'date', 'sales', 'd', 'wm_yr_wk', 'weekday']
train_cols = [c for c in dt.columns if c not in ignore_cols]
X = dt[train_cols]
Y = dt['sales']

In [ ]:
valid_size = int(X.shape[0] * 0.1)
np.random.seed(777)

valid_idx = np.random.choice(X.index.values, valid_size, replace=False)
train_idx = np.setdiff1d(X.index.values, valid_idx)
assert valid_idx.size + train_idx.size == X.shape[0]

In [ ]:
train_data = lgb.Dataset(X.loc[train_idx], Y.loc[train_idx], categorical_feature=cat_feats, free_raw_data=False)
valid_data = lgb.Dataset(X.loc[valid_idx], Y.loc[valid_idx], categorical_feature=cat_feats, free_raw_data=False)

In [ ]:
del dt, X, Y, valid_idx, train_idx
gc.collect()

## Training

In [ ]:
lgb_params = {
        "objective" : "poisson",
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
        "nthread" : 12,
        "metric": ["rmse"],
        'verbosity': 1,
        'num_leaves': 128,
        "min_data_in_leaf": 100,
}

In [ ]:
%%time
m_lgb = lgb.train(lgb_params, train_data, valid_sets=[train_data, valid_data], verbose_eval=20, categorical_feature=cat_feats, 
                  num_boost_round=1200)

In [ ]:
m_lgb.save_model(str(path/'m5_model.lgb'))

In [ ]:
m_lgb = lgb.Booster(model_file=str(path/'m5_model.lgb'))

## Prediction

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
alphas = [1.028, 1.023, 1.018]
weights = [1 / len(alphas)] * len(alphas)
assert sum(weights) == 1.0
fday = datetime(2016, 4, 25) 
assert datetime(2011, 1, 29) + timedelta(days=1914 - 1) == fday

In [ ]:
cols = [f'F{i}' for i in range(1, pred_days + 1)]
sub = pd.DataFrame()

for icount, (alpha, weight) in tqdm(enumerate(zip(alphas, weights)), total=len(alphas)):
    te = read_dt(False)
    for tdelta in tqdm(range(0, pred_days), total=pred_days):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_features(tst)
        tst = tst.loc[tst.date == day, train_cols]
        te.loc[te.date == day, 'sales'] = alpha * m_lgb.predict(tst) # magic multiplier by kyakovlev
    
    te_sub = te.loc[te.date >= fday, ['id', 'sales']].copy()
    te_sub['F'] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()['sales'][cols]
    te_sub.fillna(0., inplace=True)
    te_sub.sort_values(["id"], inplace=True)
    te_sub.reset_index(drop=False, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    
    if icount == 0:
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols] * weight

sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

In [ ]:
!wc -l submission.csv

In [ ]:
pd.read_csv('submission.csv')